In [ ]:
knitr::opts_chunk$set(message=FALSE, tidy.opts=list(width.cutoff=50), tidy=TRUE) 
def.chunk.hook  <- knitr::knit_hooks$get("chunk")
knitr::knit_hooks$set(chunk = function(x, options) {
  x <- def.chunk.hook(x, options)
  paste0("\n \\", "footnotesize","\n\n", x, "\n\n \\normalsize")
})


In [ ]:
# knitting markdown files (aka creating a pdf with code outputs) requires installing a few packages in advance.
# install.packages("rmarkdown)
# install.packages("formatR")


## Home sales: Exploratory data analysis (EDA)

Our first example is a data set about home sales in Florida. Our goal is to understand how certain covariables influence the selling price (in thousands of dollars) of a house. The covariables are size of house (in square feet), annual property tax bill (in dollars), number of bedrooms, number of bathrooms, and whether the house is new. We treat them as a random sample of a conceptual population of home sales. We first want to get to know the data a bit. Download the data set and store it in a `data.frame`.


In [ ]:
houses <- read.table("https://www.stat.ufl.edu/~aa/glm/data/Houses.dat", header=TRUE)
dim(houses)
class(houses)
names(houses)
head(houses)


The last three commands show that there are 100 rows and 7 columns, prints the column names and displays the first 6 entries. The first column (`case`) is only the index. Let us remove it.



In [ ]:
houses <- subset(houses, select=-case)
head(houses)


We also notice that `taxes`, `price` and `size` are continuous variables (data types `int` and `dbl` for integer and double), while `baths`, `beds` are on the ordinal scale (still encoded as `int`), and `new` is a categorical variable taking values `0` and `1`. In `R` we can declare categorical variables explicitly as `factors` to simplify processing them down the line (no need to create dummy variables for the `lm` method). Notice the outputs of the following commands.



In [ ]:
houses$new <- factor(houses$new)
levels(houses$new)
head(houses)


The type of `new` has changed to `fctr`. To get an idea of the range and concentration of the variables consider the following summary statistics. For continuous variables this can also be visualised by a boxplot.



In [ ]:
summary(houses)
cont_vars <- c("taxes", "price", "size")
boxplot(houses[cont_vars])


We notice that only 11 houses were new, which makes the impact of this variable unclear. There are also some extreme outliers for `taxes`, `price` and `size`. The relationship between variables can be determined by correlation and scatter plots. 



In [ ]:
cor(houses[cont_vars])
my_cols <- c("brown", "blue")
pairs(houses[cont_vars], pch=19, cex=0.5, 
      col=my_cols[houses$new],
      lower.panel=panel.smooth)
par(mfrow=c(1,3))
hist(houses$taxes); hist(houses$price); hist(houses$size)


We notice that the three continuous variables are all strongly correlated. This seems reasonable, as the annual taxes should depend on the size and a larger house costs more. Strong correlations may be a problem for linear regression. The scatter plots confirm the strong correlations (larger houses cost more) and suggest that newer houses tend to have higher prices/taxes/sizes. To compare continuous and discrete/ordinal variables a boxplot is more appropriate.



In [ ]:
par(mfrow=c(1,3))
bed_cols <- rainbow(4)
boxplot(price ~ beds, data = houses, col = bed_cols)
boxplot(taxes ~ beds, data = houses, col = bed_cols)
boxplot(size ~ beds, data = houses, col = bed_cols)


The boxplots suggest an approximate linear dependence on the number of beds. The above plots can also be combined into one plot using another package (you do not need to understand how to use this package).



In [ ]:
# install.packages("GGally")
library("GGally")
# copy data frame to ensure original data is not modified
houses_factor_beds <- data.frame(houses) 
# turn beds into factor
houses_factor_beds$beds <- factor(houses_factor_beds$beds) 
ggpairs(data=houses_factor_beds[c("taxes", "price", "size", "beds")],
        aes(color=houses_factor_beds$new), 
        diag=list(continuous=wrap("densityDiag", alpha=0.5)),
        progress=FALSE
        )


**Question 1.** *Are there any obvious outliers?* 

## Linear regression

We consider only a simple intuitive model: regress the selling price on `size` and `new`. The `summary` function describes the result.


In [ ]:
houses.lm <- lm(price ~ size + new, data=houses)
summary(houses.lm)


Alternatively, `lm(price ~., data = houses[c("price", "size", "new")])` regresses `price` on all variables in `data`. The interpretation of the estimate for `size`, for example, is that for every additional unit the mean selling price increases by 116 dollars when adjusting for the variable `new` (i.e., keeping it fixed). For a new house the mean selling price is 57.740 dollars higher (but this estimate is based on only 11 new houses and therefore rather imprecise). The $R^2$ value is large (0.72). 

Make sure you understand the output of the linear model (e.g., by studying `summary.lm` or literature), and try to answer the following questions.

**Question 2.** *How are the coefficients estimated? How are the standard errors estimated?*

**Question 3.** *What do the individual t-tests test for? How are the t-values computed? How is the p-value computed?*

**Question 4.** *What is the `residual standard error`? What is the `Multiple R-squared` value?* 

**Question 5.** *What does the F-statistic test for? How is the p-value computed?*

## Model fit

In order to check if the assumptions of the normal linear model are met, let us look at some diagnostic plots. 


In [ ]:
par(mfrow = c(2,2))
plot(houses.lm, which = c(1,2,3,4))


We obtain four plots (check the help of the `plot.lm` method with `?plot.lm` for details):

1. *Fitted values vs. residuals*: Plots the fitted/predicted values $\hat \mu:=X\hat\beta$ against the standardised residuals $\tilde{R}_i=R_i/(\hat{\sigma}\sqrt{1-H_{ii}})$. If $\mathbb{E}[\varepsilon_i|X_i]=0$, then we should see no obvious trend. Here, there is some indication that larger selling prices lead to larger variation in the residuals.
2. *Normal Q-Q plot of standardised residuals*: If the normal linear model holds true, we expect the standardised residual $\tilde{R}_i$ to be close to $\varepsilon_i$, which follows a standard normal distribution. When ordered, the sample quantiles of the standardised residuals should then follow approximately the normal quantiles. Higher departure of extreme order statistics from their expected values in Q-Q plots are to be expected. Here, there is some indication that the residuals are right-skewed (check also with `hist(houses.lm$residuals)`).
3. *Scale-location plot*: This shows the square root of the absolute value of the standardised residuals $\sqrt{|\tilde{R}_i|}$ plotted against the fitted values $\hat{\mu}$. Under homoscedacity we have approximately $\sqrt{|\tilde{R}_i|}\approx \sqrt{\operatorname{Var}(\varepsilon_i|X_i)}/\sigma=1$. Any strong trend (typically upwards) away from the constant line at `1` is indicative of heteroscedacity. Again, we see some indication that the variances are not constant. 
4. *Cook's distance plot*: This plots Cook's distance $D_i$ for the ith observation. Heuristically, one would go back to the data set to check if a point is an outlier if its Cook's distance is above 1 (cf. Agresti, Section 2.5.5). However, it is typically a bad idea to tamper with data simply because it does not follow the model we want to fit. We should only consider removing a potential outlier data point if there is suitable scientific justification (perhaps faulty measurement, different population etc.) for doing so. 
	
The first and third plots apply also directly to non-normal linear models, the other two require normally distributed errors. According to Cook's distance observation 64 is an outlier. It seems to be indeed rather different from the population and is clearly visible in the scatterplots.	


In [ ]:
rbind(houses[64,cont_vars],colMeans(houses[cont_vars]))



Another way to address visually the question if this observation is influential is to fit the same linear model without it. 



In [ ]:
houses2 = houses[-64,]
houses2.lm <- lm(price ~ size + new, data=houses2)
summary(houses2.lm)


The mean selling price for new a house is reduced from $57,736 to $41,306, the effect of size has increased , and $R^2$ has increased considerably. We conclude that this observation is indeed influential, but we will see in a later practical that in an alternative model, which allows for variances growing with the mean price, it will not be influential.

It is likely that the effects of `size` and `new` on `price` are not truly additive. We can test for this by fitting another linear model including an interaction term and performing an ANOVA to compare the models with a suitable F-test.


In [ ]:
houses2.lm2 <- lm(price ~ size + new + size:new, data=houses2)
summary(houses2.lm2)
anova(houses2.lm, houses2.lm2)


Note that the same result can be obtained in `R` by the call `lm(price ~  size*new, data=houses2)` which automatically inserts all single variables (main effects, here `size` and `new`)for every interaction term. We notice that the $R^2$-value has increased slightly (practically possibly not significant), as have the standard errors for `size` and `new`. 

**Question 6.** *What is the F-test performed in the ANOVA? How do you interpret the interaction effect and the ANOVA result?* 

We might expect that the number of bedrooms is an important predictor of selling price, yet it was not included in the above model. Does it help to include `beds` in the model?

**Question 7.** *Use the `houses` data set. Compare the two linear models (with intercept) when regressing the selling price only on the number of beds and on `size`, `new` and `beds.` Explain what you observe.*

## Answers to questions

**Question 1.** *Are there any obvious outliers?*

We see three possible outliers having large size:


In [ ]:
houses[houses$size>3500,]



Of these three only observation 64 seems to be significantly different from the other observations by having a relatively low selling price for a very large, but old house. Since there are only 11 new houses, it is not clear if this has an effect or not.

**Question 2.** *How are the coefficients estimated? How are the standard errors estimated?* 

The $\beta$ coefficient is estimated by the least squares estimator (which is the MLE here). The standard errors are $se(\hat{\beta}_j) = \hat{\sigma}(X^\top X)^{-1/2}_{jj}$ for $j=1,\dots,p$, where $$\hat{\sigma}^2 = \frac{1}{n-p}\lVert Y-X\hat{\beta}\rVert^2.$$ These quantities can be computed 'by hand' using the following code:


In [ ]:
X <- model.matrix(~ size + new, data = houses)  
Y <- houses$price
print("betahat")
betahat <- solve(t(X)%*%X, t(X)%*%Y)
betahat
Yhat <- X%*%betahat
RSS <- norm(Y - Yhat, type="F")^2
sigmahat <- sqrt(RSS/(length(Y)-length(betahat)))
print("standard errors")
sqrt(diag(solve(t(X)%*%X))) * sigmahat 


**Question 3.** *What do the individual t-tests test for? How are the t-values computed? How is the p-value computed?*

They test for the null hypothesis $H_0:\beta_j=0$ that the jth coefficient is zero. The corresponding t-statistic is $$\frac{\hat{\beta}_j}{\hat{\sigma}(X^\top X)^{-1/2}_{jj}}.$$Under the null it has a $t_{n-p}$-distribution. The p-value is the probability (under the $|t_{n-p}|$-distribution) to see a more extreme result than the observed one (the absolute value of the t-statistic), i.e., $p=P(|t_{n-p}|\geq |t|)$. By symmetry this means $p=2p_u$, where $p_u=1-pt(|t|,100-4)$ is the usual upper-tailed p-value of the t-distribution (`pt` is the distribution function of the t-distribution). A small p-value indicates significance (rejecting the null hypothesis that the variable is not in the model).

**Question 4.** *What is the residual standard error? What is the Multiple R-squared value?* 

The residual standard error is $\hat{\sigma}$ from Question 2. Multiple R-squared is the  $R^2$ statistic, indicating the amount of variability explained by the covariates included in the linear model. Higher values are better. From the output, $R^2=0.7226$ indicates a reduction of 72% in the sum of squared errors from using this prediction equation instead of $\bar{y}$, the estimate for the only parameter in the intercept-only model to predict the record times. The multiple R-squared value adjusts for the number of parameters involved.

**Question 5.** *What does the F-statistic test for? How is the p-value computed?*

It is the global test for the null hypothesis $H_0:\beta_1=\dots=\beta_{p-1}=0$ (intercept-only model) stating that none of the explanatory variables are truly correlated with the response. It therefore compares the two nested models $P_0$ and $P_1$ with $P_0$ being the null-model (1 parameter) and $P_1$ the model in question with $p=3$ parameters. The corresponding F-statistic follows a $F_{p-1,n-p}=F_{3-1,100-3}$ distribution. The p-value is the probability (under the F-distribution) to see a more extreme result than the observed one (the value of the F-statistic), i.e., $p=1-F_{p-1,n-p}(F)$. We usually expect a small p-value.  

**Question 6.** *What is the F-test performed in the ANOVA? How do you interpret the interaction effect and the ANOVA result?* 

The F-test is $$F=\frac{p_1-p_0}{n-p_1}\cdot\frac{\lVert Y-\hat{\mu}_0\rVert^2 - \lVert Y-\hat{\mu}_1\rVert^2}{\lVert Y-\hat{\mu}_1\rVert^2}\sim F_{4-3,100-4}.$$ The effect of `size` on `price` is an increase of 123\$ per square feet for older houses and 123+43\$ for newer houses. The p-value is $p=1-pf(4.448,1,96)=0.038$ and we interpret this as significant improvement of the larger model over the first model at the 5% significance level.

**Question 7.** *Use the `houses` data set. Compare the two linear models (with intercept) when regressing the selling price only on the number of beds and on `size`, `new` and `beds.` Explain what you observe.*

Let's add the variable:


In [ ]:
summary(lm(price ~ beds, data=houses))
summary(lm(price ~ size + new + beds, data=houses))
cor(houses$beds, houses$price)



There is some amount of correlation between `beds` and `price`, and `beds` is highly significant on its own when it is the only covariate. Together with `size` and `new` this effect is reversed and `beds` is not partially significant anymore. Even more, its effect size is reversed (before 61.25 and after -7.29), which illustrates *Simpson's paradox*. Note that in the second model the adjusted R-squared is smaller than in `houses.lm`, which did not include `beds` (0.7155 vs. 0.7169).
